In [ ]:
# Importação das bibliotecas
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

# Configurações de visualização
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')
%matplotlib inline

# Configurações do pandas
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.float_format', '{:.2f}'.format)

## 1.1 Carregamento dos Dados

In [ ]:
# Carregar os datasets
print("Carregando dados...")
airlines = pd.read_csv('airlines.csv')
airports = pd.read_csv('airports.csv')
flights = pd.read_csv('flights.csv')

print(f"Airlines: {airlines.shape}")
print(f"Airports: {airports.shape}")
print(f"Flights: {flights.shape}")

## 1.2 Visão Geral dos Dados

In [ ]:
# Informações sobre companhias aéreas
print("=" * 50)
print("COMPANHIAS AÉREAS")
print("=" * 50)
print(airlines.head(10))
print(f"\nTotal de companhias: {len(airlines)}")

In [ ]:
# Informações sobre aeroportos
print("=" * 50)
print("AEROPORTOS")
print("=" * 50)
print(airports.head(10))
print(f"\nTotal de aeroportos: {len(airports)}")
print(f"\nEstados únicos: {airports['STATE'].nunique()}")
print(f"\nPrincipais estados:")
print(airports['STATE'].value_counts().head(10))

In [ ]:
# Informações sobre voos
print("=" * 50)
print("VOOS")
print("=" * 50)
print(flights.head())
print(f"\nInformações do dataset:")
print(flights.info())

## 1.3 Estatísticas Descritivas

In [ ]:
# Estatísticas descritivas gerais
print("Estatísticas Descritivas - Variáveis Numéricas")
print("=" * 80)
flights.describe()

In [ ]:
# Estatísticas de atraso
print("\nEstatísticas de Atraso")
print("=" * 80)

delay_cols = [col for col in flights.columns if 'DELAY' in col]
if delay_cols:
    print(flights[delay_cols].describe())
    
    # Percentuais de atraso
    for col in delay_cols:
        if flights[col].notna().sum() > 0:
            delayed = (flights[col] > 0).sum()
            total = flights[col].notna().sum()
            pct = (delayed / total) * 100
            print(f"\n{col}: {delayed:,} voos atrasados de {total:,} ({pct:.2f}%)")

## 1.4 Análise de Valores Ausentes

In [ ]:
# Valores ausentes
print("Análise de Valores Ausentes")
print("=" * 80)

missing = pd.DataFrame({
    'Coluna': flights.columns,
    'Missing': flights.isnull().sum(),
    'Percentual': (flights.isnull().sum() / len(flights)) * 100
}).sort_values('Missing', ascending=False)

missing = missing[missing['Missing'] > 0]
print(missing)

# Visualização
if len(missing) > 0:
    plt.figure(figsize=(12, 6))
    sns.barplot(data=missing.head(20), x='Percentual', y='Coluna')
    plt.title('Top 20 Colunas com Valores Ausentes', fontsize=14, fontweight='bold')
    plt.xlabel('Percentual de Valores Ausentes (%)')
    plt.tight_layout()
    plt.show()

## 1.5 Análise Temporal

In [ ]:
# Análise por mês, dia da semana, etc.
print("Análise Temporal dos Voos")
print("=" * 80)

# Verificar colunas de tempo disponíveis
time_cols = ['MONTH', 'DAY', 'DAY_OF_WEEK', 'YEAR']
available_time_cols = [col for col in time_cols if col in flights.columns]

for col in available_time_cols:
    print(f"\nDistribuição de voos por {col}:")
    print(flights[col].value_counts().sort_index())

In [ ]:
# Visualização temporal
if 'MONTH' in flights.columns:
    fig, axes = plt.subplots(2, 2, figsize=(16, 10))
    
    # Voos por mês
    flights_by_month = flights['MONTH'].value_counts().sort_index()
    axes[0, 0].bar(flights_by_month.index, flights_by_month.values)
    axes[0, 0].set_title('Distribuição de Voos por Mês', fontweight='bold')
    axes[0, 0].set_xlabel('Mês')
    axes[0, 0].set_ylabel('Número de Voos')
    axes[0, 0].grid(True, alpha=0.3)
    
    # Voos por dia da semana
    if 'DAY_OF_WEEK' in flights.columns:
        flights_by_dow = flights['DAY_OF_WEEK'].value_counts().sort_index()
        axes[0, 1].bar(flights_by_dow.index, flights_by_dow.values, color='orange')
        axes[0, 1].set_title('Distribuição de Voos por Dia da Semana', fontweight='bold')
        axes[0, 1].set_xlabel('Dia da Semana (1=Segunda)')
        axes[0, 1].set_ylabel('Número de Voos')
        axes[0, 1].grid(True, alpha=0.3)
    
    # Atrasos por mês (se disponível)
    if 'ARRIVAL_DELAY' in flights.columns:
        avg_delay_month = flights.groupby('MONTH')['ARRIVAL_DELAY'].mean()
        axes[1, 0].plot(avg_delay_month.index, avg_delay_month.values, marker='o', linewidth=2)
        axes[1, 0].set_title('Atraso Médio por Mês', fontweight='bold')
        axes[1, 0].set_xlabel('Mês')
        axes[1, 0].set_ylabel('Atraso Médio (minutos)')
        axes[1, 0].grid(True, alpha=0.3)
    
    # Atrasos por dia da semana (se disponível)
    if 'DAY_OF_WEEK' in flights.columns and 'ARRIVAL_DELAY' in flights.columns:
        avg_delay_dow = flights.groupby('DAY_OF_WEEK')['ARRIVAL_DELAY'].mean()
        axes[1, 1].plot(avg_delay_dow.index, avg_delay_dow.values, marker='s', 
                       linewidth=2, color='green')
        axes[1, 1].set_title('Atraso Médio por Dia da Semana', fontweight='bold')
        axes[1, 1].set_xlabel('Dia da Semana (1=Segunda)')
        axes[1, 1].set_ylabel('Atraso Médio (minutos)')
        axes[1, 1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

## 1.6 Análise por Companhia Aérea

In [ ]:
# Análise por companhia
if 'AIRLINE' in flights.columns:
    # Merge com nomes das companhias
    flights_airlines = flights.merge(airlines, left_on='AIRLINE', right_on='IATA_CODE', how='left')
    
    print("Top 10 Companhias Aéreas por Número de Voos")
    print("=" * 80)
    top_airlines = flights_airlines['AIRLINE_y'].value_counts().head(10)
    print(top_airlines)
    
    # Visualização
    plt.figure(figsize=(12, 6))
    top_airlines.plot(kind='barh')
    plt.title('Top 10 Companhias Aéreas por Número de Voos', fontsize=14, fontweight='bold')
    plt.xlabel('Número de Voos')
    plt.ylabel('Companhia Aérea')
    plt.tight_layout()
    plt.show()

In [ ]:
# Atrasos por companhia
if 'AIRLINE' in flights.columns and 'ARRIVAL_DELAY' in flights.columns:
    delay_by_airline = flights_airlines.groupby('AIRLINE_y').agg({
        'ARRIVAL_DELAY': ['mean', 'median', 'std', 'count']
    }).round(2)
    delay_by_airline.columns = ['Atraso_Médio', 'Atraso_Mediano', 'Desvio_Padrão', 'Total_Voos']
    delay_by_airline = delay_by_airline.sort_values('Atraso_Médio', ascending=False)
    
    print("\nAtrasos por Companhia Aérea")
    print("=" * 80)
    print(delay_by_airline.head(10))
    
    # Visualização
    fig, axes = plt.subplots(1, 2, figsize=(16, 6))
    
    top_10_delay = delay_by_airline.head(10)
    axes[0].barh(range(len(top_10_delay)), top_10_delay['Atraso_Médio'])
    axes[0].set_yticks(range(len(top_10_delay)))
    axes[0].set_yticklabels(top_10_delay.index)
    axes[0].set_title('Top 10 Companhias com Maior Atraso Médio', fontweight='bold')
    axes[0].set_xlabel('Atraso Médio (minutos)')
    axes[0].invert_yaxis()
    
    bottom_10_delay = delay_by_airline.tail(10)
    axes[1].barh(range(len(bottom_10_delay)), bottom_10_delay['Atraso_Médio'], color='green')
    axes[1].set_yticks(range(len(bottom_10_delay)))
    axes[1].set_yticklabels(bottom_10_delay.index)
    axes[1].set_title('Top 10 Companhias com Menor Atraso Médio', fontweight='bold')
    axes[1].set_xlabel('Atraso Médio (minutos)')
    axes[1].invert_yaxis()
    
    plt.tight_layout()
    plt.show()

## 1.7 Análise por Aeroporto

In [ ]:
# Análise de aeroportos de origem
if 'ORIGIN_AIRPORT' in flights.columns:
    print("Top 15 Aeroportos de Origem por Número de Voos")
    print("=" * 80)
    top_origins = flights['ORIGIN_AIRPORT'].value_counts().head(15)
    print(top_origins)
    
    # Merge com informações dos aeroportos
    origin_info = flights.merge(airports, left_on='ORIGIN_AIRPORT', 
                                right_on='IATA_CODE', how='left', suffixes=('', '_origin'))
    
    top_origin_names = origin_info.groupby('AIRPORT').size().sort_values(ascending=False).head(15)
    
    plt.figure(figsize=(12, 6))
    top_origin_names.plot(kind='barh')
    plt.title('Top 15 Aeroportos de Origem', fontsize=14, fontweight='bold')
    plt.xlabel('Número de Voos')
    plt.ylabel('Aeroporto')
    plt.tight_layout()
    plt.show()

In [ ]:
# Atrasos por aeroporto de origem
if 'ORIGIN_AIRPORT' in flights.columns and 'ARRIVAL_DELAY' in flights.columns:
    delay_by_origin = origin_info.groupby('AIRPORT').agg({
        'ARRIVAL_DELAY': ['mean', 'median', 'count']
    }).round(2)
    delay_by_origin.columns = ['Atraso_Médio', 'Atraso_Mediano', 'Total_Voos']
    
    # Filtrar aeroportos com pelo menos 1000 voos
    delay_by_origin = delay_by_origin[delay_by_origin['Total_Voos'] >= 1000]
    delay_by_origin = delay_by_origin.sort_values('Atraso_Médio', ascending=False)
    
    print("\nAeroportos Mais Críticos (Maior Atraso Médio)")
    print("=" * 80)
    print(delay_by_origin.head(15))
    
    # Visualização
    fig, axes = plt.subplots(1, 2, figsize=(16, 8))
    
    top_delay = delay_by_origin.head(15)
    axes[0].barh(range(len(top_delay)), top_delay['Atraso_Médio'], color='red')
    axes[0].set_yticks(range(len(top_delay)))
    axes[0].set_yticklabels(top_delay.index)
    axes[0].set_title('15 Aeroportos Mais Críticos (Maior Atraso)', fontweight='bold')
    axes[0].set_xlabel('Atraso Médio (minutos)')
    axes[0].invert_yaxis()
    
    bottom_delay = delay_by_origin.tail(15)
    axes[1].barh(range(len(bottom_delay)), bottom_delay['Atraso_Médio'], color='green')
    axes[1].set_yticks(range(len(bottom_delay)))
    axes[1].set_yticklabels(bottom_delay.index)
    axes[1].set_title('15 Aeroportos Mais Pontuais (Menor Atraso)', fontweight='bold')
    axes[1].set_xlabel('Atraso Médio (minutos)')
    axes[1].invert_yaxis()
    
    plt.tight_layout()
    plt.show()

## 1.8 Análise de Distância e Tempo de Voo

In [ ]:
# Análise de distância
if 'DISTANCE' in flights.columns:
    print("Estatísticas de Distância dos Voos")
    print("=" * 80)
    print(flights['DISTANCE'].describe())
    
    fig, axes = plt.subplots(1, 2, figsize=(16, 5))
    
    # Distribuição de distâncias
    axes[0].hist(flights['DISTANCE'].dropna(), bins=50, edgecolor='black')
    axes[0].set_title('Distribuição de Distâncias dos Voos', fontweight='bold')
    axes[0].set_xlabel('Distância (milhas)')
    axes[0].set_ylabel('Frequência')
    axes[0].grid(True, alpha=0.3)
    
    # Relação distância x atraso
    if 'ARRIVAL_DELAY' in flights.columns:
        # Criar bins de distância
        flights['DISTANCE_BIN'] = pd.cut(flights['DISTANCE'], bins=10)
        delay_by_distance = flights.groupby('DISTANCE_BIN')['ARRIVAL_DELAY'].mean()
        
        axes[1].plot(range(len(delay_by_distance)), delay_by_distance.values, 
                    marker='o', linewidth=2)
        axes[1].set_title('Atraso Médio por Faixa de Distância', fontweight='bold')
        axes[1].set_xlabel('Faixa de Distância')
        axes[1].set_ylabel('Atraso Médio (minutos)')
        axes[1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

## 1.9 Análise de Correlação

In [ ]:
# Matriz de correlação
# Selecionar apenas colunas numéricas relevantes
numeric_cols = flights.select_dtypes(include=[np.number]).columns.tolist()

# Remover colunas com muitos NaN ou que não são relevantes
cols_to_correlate = [col for col in numeric_cols if flights[col].notna().sum() > len(flights) * 0.5]

if len(cols_to_correlate) > 1:
    correlation_matrix = flights[cols_to_correlate].corr()
    
    plt.figure(figsize=(14, 12))
    sns.heatmap(correlation_matrix, annot=True, fmt='.2f', cmap='coolwarm', 
                center=0, square=True, linewidths=1)
    plt.title('Matriz de Correlação - Variáveis Numéricas', fontsize=16, fontweight='bold')
    plt.tight_layout()
    plt.show()
    
    # Correlações com atraso
    if 'ARRIVAL_DELAY' in cols_to_correlate:
        print("\nCorrelações com ARRIVAL_DELAY:")
        print("=" * 80)
        delay_corr = correlation_matrix['ARRIVAL_DELAY'].sort_values(ascending=False)
        print(delay_corr)

## 1.10 Tratamento de Valores Ausentes

In [ ]:
# Estratégia de tratamento
print("Estratégia de Tratamento de Valores Ausentes")
print("=" * 80)

# Criar cópia para tratamento
flights_clean = flights.copy()

# 1. Colunas com muitos missing (>50%) - avaliar se devem ser removidas
high_missing = missing[missing['Percentual'] > 50]['Coluna'].tolist()
print(f"\nColunas com >50% de missing values: {len(high_missing)}")
print(high_missing)

# 2. Para colunas de delay: missing pode significar "sem atraso" (voo cancelado ou dados não coletados)
delay_columns = [col for col in flights.columns if 'DELAY' in col]
print(f"\nColunas de delay: {delay_columns}")

# 3. Verificar se há padrão nos missings
if 'CANCELLED' in flights.columns:
    print(f"\nVoos cancelados: {flights['CANCELLED'].sum():,}")
    print(f"Proporção de cancelados: {(flights['CANCELLED'].sum() / len(flights)) * 100:.2f}%")

if 'DIVERTED' in flights.columns:
    print(f"\nVoos desviados: {flights['DIVERTED'].sum():,}")
    print(f"Proporção de desviados: {(flights['DIVERTED'].sum() / len(flights)) * 100:.2f}%")

print("\n" + "=" * 80)
print("Dataset limpo salvo como 'flights_clean'")
print(f"Shape original: {flights.shape}")
print(f"Shape após limpeza: {flights_clean.shape}")

## 1.11 Principais Insights da EDA

### Resumo dos Insights:

1. **Volume de Dados**: 
   - Análise de milhões de registros de voos
   - Múltiplas companhias aéreas e aeroportos

2. **Padrões Temporais**:
   - Variação sazonal no número de voos
   - Dias da semana com mais/menos voos
   - Tendências de atraso por período

3. **Companhias Aéreas**:
   - Identificação das companhias com mais operações
   - Comparação de desempenho em pontualidade

4. **Aeroportos Críticos**:
   - Aeroportos com maiores volumes de tráfego
   - Identificação de hubs problemáticos

5. **Correlações**:
   - Relações entre variáveis e atrasos
   - Fatores que mais influenciam a pontualidade

### Próximos Passos:
1. Modelagem supervisionada para predição de atrasos
2. Clusterização para identificar perfis de aeroportos/rotas
3. Análises geográficas com mapas interativos
4. Feature engineering para melhorar os modelos

In [ ]:
# Salvar dados limpos para uso nos próximos notebooks
print("Salvando dados processados...")
flights_clean.to_csv('flights_processed.csv', index=False)
print("✓ Dados salvos em 'flights_processed.csv'")